In [6]:
import json 

def setPhoneName(s): 
    if s == "": return "Unknown" 
    elif "," in s: return s.replace(",", " or") 
    else: return s 
    
src = "/user/student/author_phone.json"
phoneRDD = sc.wholeTextFiles(src).flatMap(lambda tup: json.loads(tup[1]))\
        .map(lambda kv: (kv.get("author_id", None), kv.get("phone_model", None)))\
        .map(lambda kv: (kv[0], setPhoneName(kv[1])))

phoneRDD.take(5)


[(1, 'Samsung Galaxy s21 Ultra'),
 (2, 'Samsung Galaxy A52s'),
 (3, 'Samsung Galaxy Z Fold3'),
 (4, 'Apple iPhone 7'),
 (5, 'Apple iPhone 11')]

In [7]:
schema = "author_id int, phone_description string"
phoneDF = spark.createDataFrame(phoneRDD,schema)
phoneDF.printSchema()
phoneDF.show(5,truncate=False)

root
 |-- author_id: integer (nullable = true)
 |-- phone_description: string (nullable = true)

+---------+------------------------+
|author_id|phone_description       |
+---------+------------------------+
|1        |Samsung Galaxy s21 Ultra|
|2        |Samsung Galaxy A52s     |
|3        |Samsung Galaxy Z Fold3  |
|4        |Apple iPhone 7          |
|5        |Apple iPhone 11         |
+---------+------------------------+
only showing top 5 rows



In [9]:
import xml.etree.ElementTree as ET
def getPosts(s):
    posts = ET.fromstring(s)
    return posts.iter("record")

def getPostID(elem):
    return elem.find("post_id").txt

def getPostLocation(elem):
    return elem.find("location").txt

In [10]:
from pyspark.sql.types import *
schema =StructType([ 
     StructField("post_id", StringType(), True), 
     StructField("lat", DoubleType(), True), 
     StructField("lon", DoubleType(), True)])

In [11]:
latlongRDD = sc\
    .wholeTextFiles("post_records/*")\
    .flatMap(lambda xmls: getPosts(xmls[1]))\
    .map(lambda element: [getPostID(element),getPostLocation(element).split(",")])\
    .map(lambda coll: [coll[0],float(coll[1][0]), float(coll[1][1])])
latlongRDD.take(5)

2024-09-21 23:10:52,438 ERROR executor.Executor: Exception in task 0.0 in stage 4.0 (TID 4)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark3/spark-3.1.2-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/usr/local/spark3/spark-3.1.2-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/local/spark3/spark-3.1.2-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/spark3/spark-3.1.2-bin-hadoop3.2/python/pyspark/rdd.py", line 1560, in takeUpToNumLeft
    yield next(iterator)
  File "/usr/local/spark3/spark-3.1.2-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/util.py", line 73, in wrapper
    return f(*args, **kwargs)
  File "/tmp/ipykernel_5048/1398335536.py", line 4, in <lambda>
  File "/tmp/ipykernel_5048

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 4.0 failed 1 times, most recent failure: Lost task 0.0 in stage 4.0 (TID 4) (192.168.1.6 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/spark3/spark-3.1.2-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/usr/local/spark3/spark-3.1.2-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/local/spark3/spark-3.1.2-bin-hadoop3.2/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/local/spark3/spark-3.1.2-bin-hadoop3.2/python/pyspark/rdd.py", line 1560, in takeUpToNumLeft
    yield next(iterator)
  File "/usr/local/spark3/spark-3.1.2-b

In [17]:
import xml.etree.ElementTree as ET
from pyspark.sql.types import *

# Function to parse posts from the XML string
def getPosts(s):
    posts = ET.fromstring(s)
    return posts.iter("record")

# Function to get post_id with error handling for missing tags
def getPostID(elem):
    post_id_elem = elem.find("post_id")
    if post_id_elem is not None and post_id_elem.text is not None:
        return post_id_elem.text
    return None

# Function to get location with error handling for missing tags
def getPostLocation(elem):
    location_elem = elem.find("location")
    if location_elem is not None and location_elem.text is not None:
        return location_elem.text
    return None

# Define schema for the resulting DataFrame
schema = StructType([ 
     StructField("post_id", StringType(), True), 
     StructField("lat", DoubleType(), True), 
     StructField("lon", DoubleType(), True)])

# RDD processing pipeline with corrected indentation
latlongRDD = sc.wholeTextFiles("post_records/*") \
    .flatMap(lambda xmls: getPosts(xmls[1])) \
    .map(lambda element: [getPostID(element), getPostLocation(element)]) \
    .filter(lambda coll: coll[0] is not None and coll[1] is not None) \
    .map(lambda coll: [coll[0], coll[1].split(",")]) \
    .filter(lambda coll: len(coll[1]) == 2) \
    .map(lambda coll: [coll[0], float(coll[1][0]), float(coll[1][1])])

# Take the first 5 records for inspection
latlongRDD.take(5)

# If you want to convert the RDD to a DataFrame, you can do so with the schema:
latlongDF = spark.createDataFrame(latlongRDD, schema)
latlongDF.show(5)


+-------+---------+---------+
|post_id|      lat|      lon|
+-------+---------+---------+
|      1|-78.67343|146.15251|
|      2| -24.8449| 71.73862|
|      3| 33.15167|  90.2584|
|      4| 78.53576|-113.2306|
|      5| 37.09904|141.78509|
+-------+---------+---------+
only showing top 5 rows



2024-09-21 23:16:26,836 WARN impl.BlockReaderFactory: I/O error constructing remote block reader.
java.nio.channels.ClosedByInterruptException
	at java.nio.channels.spi.AbstractInterruptibleChannel.end(AbstractInterruptibleChannel.java:202)
	at sun.nio.ch.SocketChannelImpl.connect(SocketChannelImpl.java:657)
	at org.apache.hadoop.net.SocketIOWithTimeout.connect(SocketIOWithTimeout.java:192)
	at org.apache.hadoop.net.NetUtils.connect(NetUtils.java:531)
	at org.apache.hadoop.hdfs.DFSClient.newConnectedPeer(DFSClient.java:2939)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.nextTcpPeer(BlockReaderFactory.java:821)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.getRemoteBlockReaderFromTcp(BlockReaderFactory.java:746)
	at org.apache.hadoop.hdfs.client.impl.BlockReaderFactory.build(BlockReaderFactory.java:379)
	at org.apache.hadoop.hdfs.DFSInputStream.getBlockReader(DFSInputStream.java:644)
	at org.apache.hadoop.hdfs.DFSInputStream.blockSeekTo(DFSInputStream.java:575)
	

In [19]:
from pyspark.sql.types import *
schema =StructType([
    StructField("post_id",StringType(),True),
    StructField("lat",DoubleType(),True),
    StructField("lon",DoubleType(),True)
])
latlongDF = spark.createDataFrame(latlongRDD, schema)
latlongDF.printSchema()
latlongDF.show(5,truncate=False)

root
 |-- post_id: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)

+-------+---------+---------+
|post_id|lat      |lon      |
+-------+---------+---------+
|1      |-78.67343|146.15251|
|2      |-24.8449 |71.73862 |
|3      |33.15167 |90.2584  |
|4      |78.53576 |-113.2306|
|5      |37.09904 |141.78509|
+-------+---------+---------+
only showing top 5 rows



In [20]:
latlongDF.write.saveAsTable("mydb.posts_latlon")

2024-09-21 23:22:00,619 WARN conf.HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
2024-09-21 23:22:00,620 WARN conf.HiveConf: HiveConf of name hive.stats.retries.wait does not exist
2024-09-21 23:22:08,753 WARN session.SessionState: METASTORE_FILTER_HOOK will be ignored, since hive.security.authorization.manager is set to instance of HiveAuthorizerFactory.
2024-09-21 23:22:08,850 WARN conf.HiveConf: HiveConf of name hive.internal.ss.authz.settings.applied.marker does not exist
2024-09-21 23:22:08,850 WARN conf.HiveConf: HiveConf of name hive.stats.jdbc.timeout does not exist
2024-09-21 23:22:08,851 WARN conf.HiveConf: HiveConf of name hive.stats.retries.wait does not exist
2024-09-21 23:22:09,133 WARN metastore.ObjectStore: Failed to get database global_temp, returning NoSuchObjectException


In [22]:
spark.sql("USE mydb")
spark.sql("SHOW TABLES").show()
spark.sql("DESC posts_latlon").show()

+--------+-----------------+-----------+
|database|        tableName|isTemporary|
+--------+-----------------+-----------+
|    mydb|      author_bday|      false|
|    mydb|          authors|      false|
|    mydb| authors_parquet2|      false|
|    mydb|             cars|      false|
|    mydb|      celebrities|      false|
|    mydb|       continents|      false|
|    mydb|        countries|      false|
|    mydb|employee_salaries|      false|
|    mydb|           orders|      false|
|    mydb|      post_latlon|      false|
|    mydb|            posts|      false|
|    mydb|     posts_latlon|      false|
|    mydb|       spark_test|      false|
|    mydb|          titanic|      false|
|    mydb|       titanic_ui|      false|
+--------+-----------------+-----------+

+--------+---------+-------+
|col_name|data_type|comment|
+--------+---------+-------+
| post_id|   string|   null|
|     lat|   double|   null|
|     lon|   double|   null|
+--------+---------+-------+

